In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os 
import glob
import sys
from datetime import datetime, timedelta
%run read_matlab_file.ipynb

read_matlab_file.ipynb loaded


In [2]:
#TODO: make the main function: convert_erg take in T0 so that the correct coefs are automatically retrived

In [3]:
#path = "/Users/andreas/phys/master/GRB/ASIM/COEFS/BGO/" #MAC
path = "/Home/siv30/wad005/master/GRB/ASIM/COEFS/BGO/" #UiB

In [4]:
coef_filenames = glob.glob(path + "*.txt")

In [5]:
dfs_list = []
for i in np.arange(0,len(coef_filenames)):
    df = pd.read_csv(coef_filenames[i],delim_whitespace=True)
    #joining date and time 
    dt_list = []
    date_arr = df["DATE"].values
    time_arr = df["TIME"].values
    for i in np.arange(0,len(date_arr)):
        dt_str = date_arr[i]+ " " + time_arr[i]
        dt = datetime.strptime(dt_str, "%d-%b-%Y %H:%M:%S")
        dt_list.append(dt)
    df["datetime"] = dt_list
    df = df.drop(columns=["DATE","TIME"])
    dfs_list.append(df)

In [6]:
dfs = {"dau_1_det_1":dfs_list[0],"dau_1_det_2":dfs_list[1],"dau_1_det_3":dfs_list[2],"dau_2_det_1":dfs_list[3],
      "dau_2_det_2":dfs_list[4],"dau_2_det_3":dfs_list[5],"dau_3_det_1":dfs_list[6],"dau_3_det_2":dfs_list[7],
      "dau_3_det_3":dfs_list[8],"dau_4_det_1":dfs_list[9],"dau_4_det_2":dfs_list[10],"dau_4_det_3":dfs_list[11]}

In [7]:
dau_det_list = []
for i in np.arange(1,5):
        for j in np.arange(1,4):
            dau_det_list.append("dau_{}_det_{}".format(i,j))

In [8]:
tus, erg, typ, adr, dau, det, date = read_matlab_file_with_BGO_data(trig_dict_HED["GRB200415"][0])
T0 = datetime.strptime(date, "%Y-%b-%d %H:%M:%S.%f")

In [9]:
def get_coefs(T0):
    """Input only T0 for the GRB event.
    Function will return a, b, c for all 12 detectors"""
    dau_det_coefs = {}
    
    for i in np.arange(1,5): #generating the output dict containing the coefs
        for j in np.arange(1,4):
            dau_det_coefs["dau_{}_det_{}".format(i,j)] = None
            
    for dau_det in dau_det_list:
        row = dfs[dau_det].iloc[dfs[dau_det].set_index('datetime').index.get_indexer([T0], method='nearest')]
        dau_det_coefs[dau_det] = {"a":row["a"].values[0], "b": row["b"].values[0], "c":row["c"].values[0]}
    return dau_det_coefs

In [10]:
dau_det_coefs = get_coefs(T0)
dau_det_coefs

{'dau_1_det_1': {'a': 0.005245, 'b': 8.4154, 'c': 64.54628},
 'dau_1_det_2': {'a': -0.002817, 'b': 22.3117, 'c': -7.48475},
 'dau_1_det_3': {'a': 0.002895, 'b': 10.5845, 'c': 33.544},
 'dau_2_det_1': {'a': 0.006566, 'b': 6.6033, 'c': 84.98722},
 'dau_2_det_2': {'a': 0.004184, 'b': 11.7314, 'c': 29.3111},
 'dau_2_det_3': {'a': 0.004798, 'b': 9.72422, 'c': 47.9505},
 'dau_3_det_1': {'a': -0.002714, 'b': 23.1905, 'c': 0.891068},
 'dau_3_det_2': {'a': -0.005435, 'b': 28.0135, 'c': -20.6767},
 'dau_3_det_3': {'a': -0.003606, 'b': 24.6944, 'c': -0.866357},
 'dau_4_det_1': {'a': 0.00306, 'b': 12.1257, 'c': 37.1393},
 'dau_4_det_2': {'a': 0.001654, 'b': 16.4206, 'c': -43.19642},
 'dau_4_det_3': {'a': 0.002252, 'b': 12.9039, 'c': 14.2497}}

In [11]:
def convert_erg(erg,T0):
    """should specify the T0 for the event"""
    
    """keV = (A*CHANNEL^2) + (B*CHANNEL) + C"""
    keV_list = []
    a_list = []
    b_list = []
    c_list = []
    dau_det_coefs = get_coefs(T0)
    
    for i in np.arange(0,len(erg)):
        dau_val = dau[i]
        det_val = det[i]
        erg_val = erg[i]
        

        a = dau_det_coefs["dau_{}_det_{}".format(dau_val,det_val)]["a"]
        a_list.append(a)
        
        b = dau_det_coefs["dau_{}_det_{}".format(dau_val,det_val)]["b"]
        b_list.append(b)
        
        c = dau_det_coefs["dau_{}_det_{}".format(dau_val,det_val)]["c"]
        c_list.append(c)
        
        keV = a*erg_val**2 + b*erg_val + c
        
        keV_list.append(keV)
        #return  np.array(a_list), np.array(b_list), np.array(c_list)
    return np.array(keV_list)

In [12]:
#print("Max energy in MeV: ", round(max(keV_arr))/1e3)

In [13]:
def create_df(tus,erg,typ,dau,det,a,b,c,keV):

    for arr in [tus,erg,dau,det,a,b,c,keV]:
        if len(arr) != len(tus):
            return print("length of arrays is not equal")
        else:
            continue
    df = pd.DataFrame({"tus":tus,"erg":erg,"typ":typ,"dau":dau,"det":det,"a":a,"b":b,"c":c,"keV":keV})
    return df

In [14]:
#df = create_df(tus,erg,typ,dau,det,a_arr,b_arr,c_arr,keV_arr)
#df.head()

In [15]:
#df.to_csv(path_or_buf="/Home/siv30/wad005/master/GRB/calibration/GRB_200415_calibration.csv",index=False)

In [16]:
print("convert_erg.ipynb loaded")

convert_erg.ipynb loaded
